In [ ]:
import os
from os.path import join as pjoin
import pandas as pd
import zipfile
import shutil
from tqdm import tqdm
from shutil import copytree

In [ ]:
#Unzip bundles_java.zip

zip_file_path = "bundles_java.zip"
extraction_directory = "bundles_java"

if not os.path.exists(extraction_directory):
    os.mkdir(extraction_directory)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

In [ ]:
import re
def getJavaClassMethodsRE(file_path):
    # Count methods
    with open(file_path, 'r', encoding="utf-8") as file:
        java_code = file.read()
    method_pattern = re.compile(r'\s*(public|private|protected|static|\s)*[\w<>,\[\]]+\s+(\w+)\s*\([^)]*\)\s*(throws\s+\w+(\s*,\s*\w+)*)?\s*\{')
    methods = method_pattern.findall(java_code)
    method_count = len(methods)

    # Count classes
    class_pattern = re.compile(r'\s*(public|private|protected|abstract|final|\s)*(\bclass\b|\binterface\b|\benum\b)\s+(\w+)(\s+extends\s+\w+)?(\s+implements\s+\w+(,\s+\w+)*)?\s*\{')
    classes = class_pattern.findall(java_code)
    class_count = len(classes)

    return method_count, class_count

In [ ]:
#Creating a dataframe of ["bundle","#java files","#classes","#methods"]
bundles=[folder for folder in os.listdir(extraction_directory) if os.path.isdir(pjoin(extraction_directory,folder)) and not folder.startswith(".")]
bundle_info=pd.DataFrame([],columns=["bundle","#java files","#classes","#methods"])
bundle_topFiles={}
for bundle in tqdm(bundles):
    codeFiles = [file for file in os.listdir(pjoin(extraction_directory,bundle))]
    methodCount=0
    classCount=0
    file_details={}
    for file in codeFiles:
        try:
            mtds,clss=getJavaClassMethodsRE(pjoin(extraction_directory,bundle,file))
            file_details[file]={}
        except Exception as e:
            print(f"error for {pjoin(bundle,file)} {e}")
            continue
        file_details[file]["methods"]=mtds
        file_details[file]["classes"]=clss
    sorted_file_details = dict(sorted(file_details.items(), key=lambda item: (-item[1]['methods'], item[1]['classes'])))
    bundle_topFiles[bundle]=list(sorted_file_details)[:10]
    methodCount+=mtds
    classCount+=clss
    bundle_info.loc[len(bundle_info)]=[bundle,len(codeFiles),classCount,methodCount]

In [ ]:
#Sorting and exporting dataframe into csv
sorted_bundle_info = bundle_info.sort_values(by=['#methods', '#java files'], ascending=[False, False])
sorted_bundle_info.to_csv("bundle_info.csv")
sorted_bundles=list(sorted_bundle_info["bundle"])
for i in range(25):
    print(i+1,sorted_bundles[i])

In [ ]:
#Open top10 files of the top25 bundles in notepad++
import subprocess
import keyboard
import pygetwindow as gw
import time
notepad_plus_plus_path = "C:\\Program Files\\Notepad++\\notepad++.exe"
for bundle in sorted_bundles[:25]:
    file_paths=[pjoin(extraction_directory,bundle,file) for file in bundle_topFiles[bundle]]
    
    # Construct the command to open files in Notepad++
    command = [notepad_plus_plus_path]
    command.extend(file_paths)

    # Use subprocess to run the command
    process = subprocess.Popen(command)
    print(bundle)
    print(bundle_topFiles[bundle])
    print()
    # Wait for the 'q' key press
    print("Press 'q' to close Notepad++ and its tabs.")
    keyboard.wait('q')
    print()
    # Close all Notepad++ windows
    for window in gw.getWindowsWithTitle("Notepad++"):
        window.close()

    # Wait for the windows to close
    time.sleep(2)

    # Kill the Notepad++ process
    process.kill()

test for loading properties, illegal character handling, parsing different data input combinations, 